# Word Based Tokenizer

- Split on some delimiter
- Most basically: split on whitespace.
- Split on punctuation and more tokens for more advanced behaviour. 

Depending on your delimiter, you will get different tokens with different meanings. 

In [67]:
python_zen = "Special cases aren't special enough to break the rules."
tokens = python_zen.split()
print(tokens)

['Special', 'cases', "aren't", 'special', 'enough', 'to', 'break', 'the', 'rules.']


We can now create IDs from this as well. For this, we create a mapping between our tokens and IDs, which we call the vocabulary. 

In [68]:
vocab = {y: x for x, y in enumerate(tokens)}
print(vocab)  # We can show the vocab

{'Special': 0, 'cases': 1, "aren't": 2, 'special': 3, 'enough': 4, 'to': 5, 'break': 6, 'the': 7, 'rules.': 8}


In [69]:
vocab['special']  # We can map tokens to IDs

3

In [76]:
[vocab[x] for x in tokens]

[0, 1, 2, 3, 4, 5, 6, 7, 8]

<!-- ![obama](images/obama.gif) -->
<img src="images/obama.gif" style="margin-left:auto;margin-right:auto;height:500px%;">

Let's take a closer look at our tokens

```python
['Special', 'cases', "aren't", 'special', 'enough', 'to', 'break', 'the', 'rules.']
```

We see `rules.` has a dot in its token. So the token `rules` will be different from `rules.`. Maybe that's not super optimal. 

**We should probably split on punctuation as well**

## Splitting on punctuation and more
We can use `NLTK` (Natural Language Toolkit), a super useful NLP package. 

In [5]:
from nltk.tokenize import wordpunct_tokenize  # Simple RegEx based tokenizer
print(wordpunct_tokenize(python_zen))

['Special', 'cases', 'aren', "'", 't', 'special', 'enough', 'to', 'break', 'the', 'rules', '.']


## Even more smartly

In [7]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/baukebrenninkmeijer/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from nltk.tokenize import word_tokenize  # Requires a download. Uses improved TreebankWordTokenizer + tricks.
print(word_tokenize(python_zen))

['Special', 'cases', 'are', "n't", 'special', 'enough', 'to', 'break', 'the', 'rules', '.']


- `are` and `aren't` are still fairly simple, since `n't` is typically appended to negate something. Part of standard contractions in English. 
- Having a rule to handle `n't` works well enough. (e.g., 
```python
.replace("n't", " n't")
```


However, this doesn't hold for other combined words. For example:

- `token` 
- `tokens`
- `tokenizer`
- `tokenization` 

all are unique tokens and get corresponding unique IDs. 

This result in **a lot** of tokens to be able to understand our text!

A common fix is to limit the vocabulary size. Typically, the $n$ most frequent words will be allowed in the vocab. 

Will render a lot of words **Out of Vocabulary (OOV)**!

OOV is typically tokenized with the `<UNK>` token.

## Usage of word tokenizer

**Word-based tokenizers** were really popular before the NLP revolution in 2017. 

Both **GloVe** and **Word2Vec** used word-based tokenization. Focus was much more on the embedding algorithm, rather than the tokenization. 

As shown, word-based tokenizers have some downsides. 

## Let's see if we can fix them!

# Character Based Tokenizers
These tokenizers are conceptually very simple. Each character is a token.

- `Small vocabulary`. For ASCII, 256. With unicode, this is max 1.1M characters.
- No *out of vocabulary* tokens.
- Lose a lot of meaninful information
- Models typically have `max input lengths` (e.g., 256 tokens)

In [77]:
list('Cat')

['C', 'a', 't']

## Mapping to a vocabulary
The vocabulary of a text will be really simple, with one index mapping to a letter. 

In [90]:
{y:x for x, y in enumerate(set(python_zen))}

{'b': 0,
 'k': 1,
 'g': 2,
 'i': 3,
 'S': 4,
 'h': 5,
 ' ': 6,
 "'": 7,
 'u': 8,
 'r': 9,
 't': 10,
 's': 11,
 'p': 12,
 'a': 13,
 'n': 14,
 'o': 15,
 'c': 16,
 '.': 17,
 'l': 18,
 'e': 19}

## Character based tokenization of Chinese characters
*Disclaimer: I do not speak a word of chinese*

- Characters have meaning by themselves
- But can change by combining characters

For example:

[馬來西亞 means Malaysia, but taken separately they mean "Horse come to Western Asia".](https://medium.com/@jjsham/nlp-tokenizing-chinese-phases-3302da4336bf).

Some good reads that use character based tokenization:
1. [Neural Machine Translation in Linear Time. Kalchbrenner et al. 2017](https://arxiv.org/pdf/1610.10099.pdf)
2. [Fully Character-Level Neural Machine Translation without Explicit Segmentation - Lee et al. 2017.](https://aclanthology.org/Q17-1026.pdf)
3. [Learning to Generate Reviews and Discovering Sentiment - Radford et al. 2017.](https://arxiv.org/pdf/1704.01444.pdf)

From these papers, you can already see that these tokenizers were mainly used a while back, around 2017. 

# Sub-word tokenizers

Subword tokenizers combine the two approaches of word and character tokenizers. 
Keep the advantages of both, limit the disadvantages.

Types:
- Byte-Pair encoding
- WordPiece
- Unigram

Special case:
- SentencePiece

## Byte Pair Encoding (BPE)

- Based on a lossless data compression algorithm, proposed by Philip Gage in 1994.
- Replace the most common pair of bytes with a byte that does not occur in the data.
- First adapted for Neural Machine Translation by [*Sennrich et al.*](https://aclanthology.org/P16-1162.pdf) in 2015.
- Used in many SOTA models: Transformer, GPT, GPT-2, XLM, FlauBERT, Roberta.


Note: BPE requires the data be split into words already. This can be whitespace splitting, or more advanced like Spacy/NLTK. This step is called pre-tokenization. We'll see why this is. 

### The Example from wikipedia
We'll encode:

```
aaabdaaabac
```

The most common pair is `aa`, which will be replace by a byte in the data that is not present, say `Z`. 

```
ZabdZabac
Z=aa
```

We repeat this step, replacing `ab` with `Y`:

```
ZYdZYac
Y=ab
Z=aa
```

The only byte-pair left is now `ac`, which only occurs once. However, you could continue recursively, replacing `ZY` with `X` if that should occur frequently.

```
XdXac
X=ZY
Y=ab
Z=aa
```

Adapting this to the NLP domain, the list of replacements (`Z`, `Y` and `X`) can now be considered your vocabulary with tokens, with the slight change that all tokens are initially in the vocabulary, and they are incrementally merged together. 

**Let's see if we have some time**

### A natural language example
*Note: example taken from the excellent [Huggingface tokenizer summary](https://huggingface.co/docs/transformers/tokenizer_summary)* 🤗.

Let's say that after pre-tokenization, we have the following set of words. With BPE, we need the words and their counts, because we want to merge based on frequency. 
```python
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```
The base vocab consequently is: `["b", "g", "h", "n", "p", "s", "u"]`

Splitting all words into vocabulary symbols, we get:

```python
("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)
```
BPE counts the frequency of all symbol pairs, and picks the most frequent. Here, we have `u` + `g` present $10+5+5=20$ times. 

Consequently, `ug` is added to the vocabulary, and we replace the `u`+`g` symbols in our words with `ug`. This results in:
```python
("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)
```

And our vocab now is: `["b", "g", "h", "n", "p", "s", "u", "ug"]`

## Hyperparameter: `vocab_size`
We continue iteratively applying this algorithms until we hit our desired vocabulary size, which is our vocab size + the number of merges. For example, GPT has a vocabulary of 40,478 build up from 478 base characters and 40.000 merges.

This learned vocabulary can now be used for unseen text. However, consider that if unknown characters are encountered they will be encoded as `<UNK>` (e.g., `mug` would become `["<UNK>", "ug"]`. 

## Dataset specific vocabulary
A tokenizer trained on the common crawl will differ from one trained on Wikipedia. For example, GPT is trained on the BooksCorpus (800M words).

**Let's have a look at the trained GPT tokenizer.**

In [41]:
from transformers import OpenAIGPTTokenizer
bpe_tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

print(len(bpe_tokenizer.get_vocab()))
bpe_tokenizer.get_vocab()

40478


{'.': 1,
 ',': 2,
 't': 3,
 'h': 4,
 'e': 5,
 '"': 6,
 'o': 7,
 'a': 8,
 'n': 9,
 'd': 10,
 'i': 11,
 'f': 12,
 'w': 13,
 's': 14,
 'y': 15,
 'u': 16,
 'r': 17,
 "'": 18,
 '?': 19,
 'm': 20,
 'b': 21,
 '-': 22,
 'v': 23,
 'p': 24,
 'c': 25,
 'l': 26,
 'k': 27,
 'j': 28,
 '!': 29,
 'g': 30,
 '*': 31,
 ';': 32,
 ':': 33,
 'x': 34,
 'q': 35,
 'z': 36,
 ')': 37,
 '(': 38,
 '1': 39,
 '/': 40,
 '_': 41,
 '2': 42,
 '3': 43,
 '4': 44,
 '~': 45,
 '5': 46,
 '#': 47,
 '0': 48,
 '6': 49,
 '7': 50,
 '$': 51,
 '>': 52,
 '9': 53,
 '8': 54,
 '[': 55,
 ']': 56,
 '<': 57,
 '&': 58,
 '%': 59,
 '¨': 60,
 '`': 61,
 'é': 62,
 '»': 63,
 '«': 64,
 '=': 65,
 '•': 66,
 '@': 67,
 '+': 68,
 '©': 69,
 '¡': 70,
 '{': 71,
 '}': 72,
 'ª': 73,
 'ñ': 74,
 'ï': 75,
 '‖': 76,
 'ç': 77,
 'í': 78,
 '^': 79,
 '£': 80,
 '§': 81,
 '♥': 82,
 '−': 83,
 'à': 84,
 '|': 85,
 '°': 86,
 '¦': 87,
 'ł': 88,
 'ĩ': 89,
 'ü': 90,
 '®': 91,
 'ù': 92,
 'á': 93,
 'â': 94,
 'ó': 95,
 'è': 96,
 '∞': 97,
 'ë': 98,
 'ä': 99,
 '♪': 100,
 'ò': 10

In [48]:
bpe_tokenizer.tokenize(python_zen)

['special</w>',
 'cases</w>',
 'are</w>',
 "n't</w>",
 'special</w>',
 'enough</w>',
 'to</w>',
 'break</w>',
 'the</w>',
 'rules</w>',
 '.</w>']

In [101]:
# Like directors
bpe_tokenizer.tokenize('M. Night Shyamalan')

['m.</w>', 'night</w>', 'shy', 'am', 'alan</w>']

In [103]:
# The fear of long words
print(bpe_tokenizer.tokenize(":Hippopotomonstrosesquippedaliophobia"))

[':</w>', 'hippo', 'po', 'tom', 'on', 'stro', 'se', 'squi', 'pp', 'ed', 'ali', 'op', 'ho', 'bia</w>']


In [99]:
# A very long Java Class name
print(bpe_tokenizer.tokenize('ProjectContractChargingPeriodProjectAccountReferenceVM'))  

['projec', 't', 'contrac', 't', 'char', 'ging', 'peri', 'od', 'projec', 't', 'accoun', 'tre', 'fer', 'en', 'c', 'ev', 'm</w>']


In [43]:
bpe_tokenizer(python_zen)

{'input_ids': [2983, 5767, 640, 538, 2983, 1046, 485, 2158, 481, 4556, 239], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Looks a lot like what we had at the start ;)

# WordPiece
- Used by: BERT, distilBERT, Electra
- Similar to BPE: starts with a base vocabulary that includes every character, but merges based on maximum likelihood of the training data, rather than just frequency. 
- Is a Google internal model without Open Source implementation. SentencePiece is Google's open source version of this. We don't really know how much they differ. 
- First proposed by [Schuster and Nakajima, 2015](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf), in the paper called *Japanese and Korean Voice Search*. Both working at Google. 

## Comparisons with other implementations

|Feature|SentencePiece|[subword-nmt](https://github.com/rsennrich/subword-nmt)|[WordPiece](https://arxiv.org/pdf/1609.08144.pdf)|
|:---|:---:|:---:|:---:|
|Supported algorithm|BPE, unigram, char, word|BPE|BPE*|
|OSS?|Yes|Yes|Google internal|
|Subword regularization|[Yes](#subword-regularization)|No|No|
|Python Library (pip)|[Yes](python/README.md)|No|N/A|
|C++ Library|[Yes](doc/api.md)|No|N/A|
|Pre-segmentation required?|[No](#whitespace-is-treated-as-a-basic-symbol)|Yes|Yes|
|Customizable normalization (e.g., NFKC)|[Yes](doc/normalization.md)|No|N/A|
|Direct id generation|[Yes](#end-to-end-example)|No|N/A|

*From the [SentencePiece Github](https://github.com/google/sentencepiece).*

## Differences with BPE
- Subwords are identified with a prefix (`##`)
- If no subword can be found in the vocabulary, the whole word is tokenized as `[UNK]`. So the previous example of encoding `mug` as `["<UNK>", "ug"]` cannot happen with wordpiece. 

## Merging rules
Search for the symbol pair with the highest score. This prioritizes combining pairs that aren't as frequent in the vocabulary. 
$$ score_{ij} = \frac{f_{ij}}{f_i \cdot f_j} $$

If $f_i \cdot f_j$ is very small, the score becomes very large. 

So say we have the same words as before:
```yaml
Corpus: ("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```

Tokenizing with the start vocabulary:
```yaml
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u"]
Corpus: ("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##g" "##s", 5)
```

1. The most frequent pair is (`##u`, `##g`), 20 times, but the individual frequency of `##u` is very high. 
2. The result is that the pair's score is only $\frac{20}{36\times20} = \frac{1}{36}$. 
3. This goes for all combinations with `##u`. 
4. The highest scoring pair is (`##g`, `##s`) - at $\frac{5}{20\times5}=\frac{1}{20}$. This is also the only pair without `##u` in it. 

First merge is (`##g`, `##s`) --> (`##gs`). 

We then get, with our new token:
```yaml
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs"]
Corpus: ("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##gs", 5)
```

And the next step, where `hu` is the token with the highest likelihood increase to the corpus. 

```yaml
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu"]
Corpus: ("hu" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)
```

Again, we apply this iteratively until we have our desired vocabulary size. 

In [49]:
from transformers import BertTokenizer
wordpiece_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

print(len(wordpiece_tokenizer.get_vocab()))
wordpiece_tokenizer.get_vocab()

28996


{'[PAD]': 0,
 '[unused1]': 1,
 '[unused2]': 2,
 '[unused3]': 3,
 '[unused4]': 4,
 '[unused5]': 5,
 '[unused6]': 6,
 '[unused7]': 7,
 '[unused8]': 8,
 '[unused9]': 9,
 '[unused10]': 10,
 '[unused11]': 11,
 '[unused12]': 12,
 '[unused13]': 13,
 '[unused14]': 14,
 '[unused15]': 15,
 '[unused16]': 16,
 '[unused17]': 17,
 '[unused18]': 18,
 '[unused19]': 19,
 '[unused20]': 20,
 '[unused21]': 21,
 '[unused22]': 22,
 '[unused23]': 23,
 '[unused24]': 24,
 '[unused25]': 25,
 '[unused26]': 26,
 '[unused27]': 27,
 '[unused28]': 28,
 '[unused29]': 29,
 '[unused30]': 30,
 '[unused31]': 31,
 '[unused32]': 32,
 '[unused33]': 33,
 '[unused34]': 34,
 '[unused35]': 35,
 '[unused36]': 36,
 '[unused37]': 37,
 '[unused38]': 38,
 '[unused39]': 39,
 '[unused40]': 40,
 '[unused41]': 41,
 '[unused42]': 42,
 '[unused43]': 43,
 '[unused44]': 44,
 '[unused45]': 45,
 '[unused46]': 46,
 '[unused47]': 47,
 '[unused48]': 48,
 '[unused49]': 49,
 '[unused50]': 50,
 '[unused51]': 51,
 '[unused52]': 52,
 '[unused53]': 53

In [105]:
print(wordpiece_tokenizer.tokenize(python_zen))

['Special', 'cases', 'aren', "'", 't', 'special', 'enough', 'to', 'break', 'the', 'rules', '.']


In [114]:
print(bpe_tokenizer.tokenize('wordpiece'))
print(wordpiece_tokenizer.tokenize('wordpiece'))

['word', 'piece</w>']
['word', '##piece']


In [112]:
# Like directors
print(bpe_tokenizer.tokenize('M. Night Shyamalan'))
print(wordpiece_tokenizer.tokenize('M. Night Shyamalan'))

['m.</w>', 'night</w>', 'shy', 'am', 'alan</w>']
['M', '.', 'Night', 'S', '##hya', '##mal', '##an']


In [110]:
# The fear of long words
print(bpe_tokenizer.tokenize(":Hippopotomonstrosesquippedaliophobia"))
print(wordpiece_tokenizer.tokenize(":Hippopotomonstrosesquippedaliophobia"))

[':</w>', 'hippo', 'po', 'tom', 'on', 'stro', 'se', 'squi', 'pp', 'ed', 'ali', 'op', 'ho', 'bia</w>']
[':', 'Hip', '##pop', '##oto', '##mons', '##tro', '##ses', '##qui', '##pped', '##ali', '##op', '##ho', '##bia']


In [115]:
# A very long Java Class name
print(bpe_tokenizer.tokenize('ProjectContractChargingPeriodProjectAccountReferenceVM'))  
print(wordpiece_tokenizer.tokenize('ProjectContractChargingPeriodProjectAccountReferenceVM'))  

['projec', 't', 'contrac', 't', 'char', 'ging', 'peri', 'od', 'projec', 't', 'accoun', 'tre', 'fer', 'en', 'c', 'ev', 'm</w>']
['Project', '##C', '##ont', '##rac', '##t', '##C', '##har', '##ging', '##P', '##eri', '##od', '##P', '##ro', '##ject', '##A', '##cco', '##unt', '##R', '##ef', '##ere', '##nce', '##V', '##M']


# Unigram
- Takes a opposite approach as BPE and WordPiece: starts with many long words and breaks them up. 
- Tries to remove words with the least impact on the loss (i.e. are least required). 
- Unigram is often used through SentencePiece
- Used in SOTA models such as: AlBERT, T5, mBART, Big Bird, and XLNet.

Let's take the same corpus again:
```yaml
corpus: ("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
vocabulary: ["h", "u", "g", "hu", "ug", "p", "pu", "n", "un", "b", "bu", "s", "hug", "gs", "ugs"]
```
The vocab is all possible subtokens, that aren't spanning the whole word. 

We see `hug` is in here, due to `hugs`. 

The unigram model sees each token independently of any token that came prior. The probability of each token given the previous context is just the probability of that token. 
The probability of a token is calculated over the total vocabulary:
$$ \mathcal{P}(t) = \frac{f_t}{\sum_{t \in T}f_t} $$

Where $T$ is the set of tokens and $f_t$ indicates the frequency of a token. 

So pretty simple: if the token `ug` is present 20 times, and the sum of all token frequencies is 210, we get: 

$$ \mathcal{P}(\text{"ug"}) = \frac{20}{210} = 0.095 $$

```yaml
corpus: ("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
vocab: ("h", 15) ("u", 36) ("g", 20) ("hu", 15) ("ug", 20) ("p", 17) ("pu", 17) ("n", 16)
("un", 16) ("b", 4) ("bu", 4) ("s", 5) ("hug", 15) ("gs", 5) ("ugs", 5)
```

These are the frequencies for each token in our vocabulary. The frequency sum is 210. 

If we want to calculate the probability of a word, we just multiple the probabilities of each token. 

$$ \mathcal{P}([\text{"p"}, \text{"u"}, \text{"g"}]) = \mathcal{P}(\text{"p"}) \times \mathcal{P}(\text{"u"}) \times \mathcal{P}(\text{"g"}) = \frac{17}{210} \times \frac{36}{210} \times \frac{20}{210} = 0.0013 $$

For each word, the chosen representation is the one with the highest probability. 

# How do we train then? 
The model is essentially overfitted to the dataset. 
1. Calculate the loss for the whole corpus, with the current vocabulary. 
2. Remove the vocabulary token whose removal increases the loss the least. 
3. We continue removing tokens until we reach our desired vocabulary size. 

Loss for one word is calculated as:

$$ \text{loss}_w = f_w \cdot -\log \mathcal{P}(w)  $$
So taking `pug` as an example:
$$
 \text{loss}_{\text{"pug"}} = 5 \cdot -\log0.0013 \\ 
 = 5 \cdot 2.88 \\
 = 14.43  $$
 
 Total loss is then calculated with:
 $$\text{loss} = \sum_{w \in W} \text{loss}_w $$

Algorithm is as follows:
1. Calculate loss for each word in the corpus with current vocab and sum.
2. Iteratively remove a token from the vocabulary.
3. Calculate loss with new vocabulary.
4. Remove the token with the lowest negative effect on the loss.
5. If vocab size > desired vocab size: go back to step 1.

# And that's how you train a unigram tokenizer!

# In conclusion

3 types of tokenizers:
- Wordbased
- Character-based
- Subword-based 

Where Subword tokenization has three different methods:
- BPE - Pair frequency based
- WordPiece/SentencePiece - Pair frequency vs token frequency based
- Unigram - Loss minimization